# Lecture 1: Rcpp

In [1]:
library(Rcpp)

## 1

In [2]:
cppFunction('
int one() {
    return 1;
}
')

x <- one()

## 2
### summing elements in vector

In [3]:
cppFunction('
double sumC(NumericVector x) {
    int n = x.size();
    double total = 0;
    int i;
    for (i = 0; i < n; i++) {
        total = total + x[i];
    }
    return total;
}')

test = 1:100
sumC(test)

[1] 5050

## 3 
### calculating simple mean of vector

In [5]:
cppFunction('
double my_mean(NumericVector x) {
    int n = x.size();
    double mean = 0;
    int i;
    for (i = 0; i < n; i++) {
        mean = mean + x[i];
    }
    mean = mean / n;
    return mean;
}')

my_mean(c(1,2,3))

[1] 2

## 4 
### vector is input and outputs a vector with elements that are >c as the same and elements that are <c are returned as 0

In [6]:
cppFunction('
NumericVector my_check_C(NumericVector x, double c) {
    int n = x.size();


    // this works the same way as python
    // clone must be used for this otherwise
    // the variable points to the same element in memory
    NumericVector out = clone(x);
    int i;

    for (i = 0; i < n; i++) {
        if (x[i] <= c) {
            out[i] = 0;
        }
    }
    return out;
}')

my_check_C(c(1,3,5,5,3,2,2,4,3,4,7,6,4,3,5,5), 3)

[1] 0 0 5 5 0 0 0 4 0 4 7 6 4 0 5 5

## 5
### calculate cumulative sum of elements in vector and return as another vector

In [7]:
cppFunction('
NumericVector my_cumsum(NumericVector x) {
    int n = x.size();

    NumericVector out = clone(x);
    double total = 0;
    int i;

    for (i = 0; i < n; i++) {
        total = total + x[i];
        out[i] = total;
    }
    return out;
}')

my_cumsum(1:10)


[1]  1  3  6 10 15 21 28 36 45 55

## 6
### takes vector, returns vector of (min,max)

In [8]:
cppFunction('
NumericVector my_range(NumericVector x) {    
    NumericVector out(2);
    out[1] = max(x);
    out[0] = min(x);
    return out;
}')

my_range(c(3,1,5,2))

[1] 1 5

### practicing

In [10]:
cppFunction('
NumericVector my_range_manual(NumericVector x) {    
    NumericVector out(2);
    NumericVector minmax(2);
    int n = x.size();
    int i;

    double maximum = 0;
    double minimum = 0;

    for (i = 0; i < n; i++) {
        minmax[0] = x[i];
        minmax[1] = x[i];
        if (minmax[0] > x[i]) {
            out[0] = x[i];
        }
        if (minmax[1] < x[i]) {
            out[1] = x[i];
        }
    }
    return out;
}')

my_range(c(3,1,5,2,2,3,4,55,6,2,4,3,2,24,1,343,5,4,23,12,32,3,23,23,332,3,2,3,2,-1))

[1]  -1 343

## 7
### standard deviation per column using sugar mean

In [11]:
cppFunction('
NumericVector rowSd_C(NumericMatrix mt) {
    int nrow = mt.nrow();
    int ncol = mt.ncol();

    NumericVector means(nrow);
    NumericVector out(nrow);
    double total;
    int i,j;

    //loop for rows
    for (i = 0; i < nrow; i++) {
        total = 0;
        //Sugar for computing the row means
        means[i] = mean(mt(i,_));
        //loop for columns
        for (j = 0; j < ncol; j++) {
            // pow is for x^y
            total = total + pow(mt(i,j) - means[i], 2.0);
        }
        out[i] = sqrt(total / (ncol-1));
    }
    return out;
}')

## 8
### same as 7 but wth more sugar

In [12]:
cppFunction('
NumericVector rowSd_C(NumericMatrix mt) {
    int nrow = mt.nrow();
    int ncol = mt.ncol();

    NumericVector out(nrow);
    double total;
    int i,j;

    //loop for rows
    for (i = 0; i < nrow; i++) {
        out[i] = sd(mt(i,_));
    }
    return out;
}')

## 9
### max,min,mean,sd for each col, summary statistics

In [13]:
cppFunction('
NumericVector summary_stats(NumericMatrix mt) {
    int nrow = mt.nrow();
    NumericMatrix out(nrow,4);
    int i;

    //loop for rows
    for (i = 0; i < nrow; i++) {
        out(i,0) = max(mt(i,_));
        out(i,1) = min(mt(i,_));
        out(i,2) = mean(mt(i,_));
        out(i,3) = sd(mt(i,_));
    }
    return out;
}')

summary_stats(matrix(c(1,3,2,4,5, 4,5,3,2,8, 1,2,3,45,5, 4,3,2,43,5, 3,4,5,56,3), nrow=5, ncol=5, byrow=T))
summary_stats(matrix(runif(30),nrow=5))

5,1,3.0,1.581139
8,2,4.4,2.302173
45,1,11.2,18.952572
43,2,11.4,17.700282
56,3,14.2,23.381617


0.7543621,0.262912092,0.4725014,0.1780015
0.9082272,0.001220338,0.3887628,0.3162586
0.9113458,0.229477662,0.4632574,0.2858648
0.7555084,0.191617840,0.4113783,0.2018169
0.8807286,0.062047761,0.5983334,0.3387061


## 10 
###  using dataframes and lists

In [14]:
cppFunction ('
List changeDataframe(DataFrame DF) {
    // we receive a DF data.frame object and access each column by name
    IntegerVector a = DF["a"];
    CharacterVector b = DF["b"];

    // We use clone (copy) so that the original dataframe is not modified
    a = clone(a);
    b = clone(b);

    // do something
    a[2] = 42;
    b[1] = "foo";

    // create a new data frame with two columns named newa and newb
    DataFrame NDF = DataFrame::create(Named("newa")=a,
    Named("newb")=b);

    // and return old and new in list with two elements: origDataFrame and newDataFrame
    return(List::create(Named("origDataFrame")=DF,
    Named("newDataFrame")=NDF));
}')


In [1]:
#include <iostream>